# PINN inversión de la ecuación de onda
Esta PINN tiene dos propositos:
* Resolver la ecuación de onda en 2 dimensiones
* Encontrar el campo de velocidades a través del cual se propaga la onda

Esto se busca incluyendo la ecuación de onda en la función de pérdida, junto con condiciones de frontera absorbentes correspondientes a la frontera en el subsuelo y condiciones de frontera libre para la superficie (Neumann iguales a cero).

Además también se incluyen los tiempos de arribo de la onda a la superficie en distintos puntos en la función de pérdida.

La ecuación de onda en dos dimensiones es:
    \begin{align}
    \frac{1}{c^2} \frac{\partial^2 u}{\partial t^2}= \frac{\partial ^2 u}{\partial x^2}+\frac{\partial ^2 u}{\partial z^2},  \ \ \ \ 0< t ,  \ \ \ 0 < x < 1, \ \ \ 0 < z < 1
    \end{align}
Con condiciones iniciales:
    \begin{align} u(x,z,0)=e^{-60(x^2+z^2)}, \ \ \ \ 0 < x < 1, \ \ \ 0< z < 1 \end{align}
    \begin{align}\frac{\partial u(x,z,0)}{\partial t}=0 \ \ 0 < x < 1, \ \ \ 0< z < 1 \end{align}
Y condiciones de frontera:
    \begin{align} \frac{\partial u}{\partial z}(x,0,t)=0, \frac{\partial u}{\partial t}(x,1,t)+c\frac{\partial u}{\partial z}(x,1,t)=0, \frac{\partial u}{\partial t}(0,z,t)+c\frac{\partial u}{\partial x}(0,z,t)=0, \frac{\partial u}{\partial t}(1,z,t)+c\frac{\partial u}{\partial x}(1,z,t)=0, \ \ t > 0 \end{align}
    
La condición de la frontera superior (z=0) es libre y el resto son absorbentes.

# ¿Qué es una PINN?

Una Red Neuronal Físicamente Informada (PINN, por sus siglas en inglés) es un tipo de red neuronal que incorpora conocimientos de leyes físicas en su proceso de aprendizaje. En este caso, estamos utilizando una PINN para resolver la ecuación de onda y realizar inversión sísmica simultáneamente.

Las PINNs tienen varias ventajas sobre los métodos numéricos tradicionales:

1. Pueden manejar dominios irregulares y condiciones de contorno complejas.
2. No requieren una malla fija.
3. Pueden incorporar datos de observación directamente en el proceso de aprendizaje.

En este notebook, veremos cómo implementar una PINN para resolver el problema descrito arriba.

In [ ]:
# Celda necesaria para correr el código en Google Colab
# import os;os.environ["TF_USE_LEGACY_KERAS"]="1"

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import timeit

# Para hacer animaciones
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
# Definición del dominio espacio-temporal
# Aquí establecemos los límites del dominio en espacio (x, z) y tiempo (t)
# También generamos puntos aleatorios dentro de este dominio para el entrenamiento

#Parametros para el dominio espacio temporal
x_min = 0 #km
x_max = 1  # Dominio en x. Inicia en 0
z_min = 0
z_max = 1  # Dominio en y. Inicia en 0
t_min = 0 #s
t_max = 6 # Tiempo total. Inicia en 0

# Se define el dominio espacio temporal
x = np.random.uniform(x_min, x_max, (10000, 1))
z = np.random.uniform(z_min, z_max, (10000, 1))
t = np.random.uniform(t_min, t_max, (10000, 1))
xzt = tf.convert_to_tensor(np.hstack((x, z, t)), dtype=tf.float32)
x = xzt[:, 0:1]
z = xzt[:, 1:2]
t = xzt[:, 2:3]

# Se añaden puntos para las condiciones iniciales
n_ic = 1000  # número de puntos que se añaden

x_ic=tf.random.uniform((n_ic, 1), 0, 1)
z_ic=tf.random.uniform((n_ic, 1), 0, 1)
t_ic=tf.zeros((n_ic, 1))

# Se añaden puntos en la frontera
n_boundary = 1000  # número de puntos que se añaden para cada una de las cuatro fronteras

# Fronteras superior e inferior
x_tb = tf.random.uniform((n_boundary, 1), 0, 1)
z_top = tf.zeros((n_boundary, 1))
z_bottom = tf.ones((n_boundary, 1))

# Fronteras izquierd y derecha
z_lr = tf.random.uniform((n_boundary, 1), 0, 1)
x_left = tf.zeros((n_boundary, 1))
x_right = tf.ones((n_boundary, 1))

# Se combinan todos los puntos de frontera
x_boundary = tf.concat([x_tb, x_tb, x_left, x_right], axis=0)
z_boundary = tf.concat([z_top, z_bottom, z_lr, z_lr], axis=0)

# Se añade un tensor de tiempo del mismo tamaño
t_boundary = tf.random.uniform((4*n_boundary, 1), t_min, t_max)

# Se combinan con los puntos originales
x = tf.concat([x, x_boundary, x_ic], axis=0)
z = tf.concat([z, z_boundary, z_ic], axis=0)
t = tf.concat([t, t_boundary, t_ic], axis=0)

# Se concatenan en un solo tensor
xzt = tf.concat([x, z, t], axis=1)

def f(x, z):
    # Condición inicial del desplazamiento (Pulso Gaussiano)
    return tf.exp(-60 * ((x-0.5)**2 + (z-0.5)**2))

def g(x, z):
    # Condición inicial de la velocidad (cero)
    return tf.zeros_like(x)

# Se cargan los datos de los sismogramas
n=20 # Se define la misma n que cuando se generaron los datos # 20 #10
x_sism=[]
u_sism=[]
for i in range(n):
    data = np.load(f"../Código Final/Datasets_Sism/x_{i+1}.npz")
    t_sism = data["t"][1:]
    x_sism.append(data["u"][0])
    u_sism.append(data["u"][1:]) 

# Arquitectura de las redes neuronales

En este problema, utilizamos dos redes neuronales:

1. **PINN**: Esta red aprende la solución de la ecuación de onda u(x,z,t). Tiene la siguiente arquitectura:
   - Entrada: 3 neuronas (x, z, t)
   - 4 capas ocultas de 50 neuronas cada una con activación tanh
   - Salida: 1 neurona (u)

2. **VelocityModel**: Esta red aprende el campo de velocidades v(x,z). Su arquitectura es:
   - Entrada: 2 neuronas (x, z)
   - 5 capas ocultas de 20 neuronas cada una con activación tanh
   - Salida: 1 neurona con activación softplus para asegurar valores positivos de velocidad

Estas arquitecturas fueron elegidas basándonos en experimentos previos y la literatura existente sobre PINNs para problemas similares.

In [ ]:
# Se define una red de [3]+[30]*3+[1]
# Con función de activación tanh
# El inicializador por defecto es Glorot uniform para los pesos y Zero para los sesgos usando tf.keras.layers.Dense
# Para u(x,z,t)
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense4 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense5 = tf.keras.layers.Dense(1, activation=None)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x) # Esto es el output u

# Se define una red de [2]+[20]*5+[1]
# Con función de activación tanh y softplus al final para asegurar un valor positivo de la velocidad
# Para v(x,z)
class VelocityModel(tf.keras.Model):
    def __init__(self):
        super(VelocityModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense4 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense5 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense6 = tf.keras.layers.Dense(1, activation='softplus') 

    def call(self, xz):
        x = self.dense1(xz)
        x = self.dense2(x) + x
        x = self.dense3(x)
        x = self.dense4(x) + x
        x = self.dense5(x) + x
        return self.dense6(x)  # Esto es el output c

# Proceso de entrenamiento

El entrenamiento de nuestra PINN se realiza minimizando una función de pérdida total que combina varios términos:

1. Pérdida de la ecuación diferencial (PDE)
2. Pérdida de las condiciones iniciales (IC)
3. Pérdida de las condiciones de frontera (BC)
4. Pérdida de los datos de sismogramas

Cada uno de estos términos tiene un peso asociado (lambda) que controla su importancia relativa en la función de pérdida total.

Utilizamos el optimizador Adam con una tasa de aprendizaje de 1e-4. El entrenamiento se realiza durante un número especificado de epochs, y cada 10,000 epochs guardamos el estado del modelo y visualizamos algunas predicciones para monitorear el progreso.

In [ ]:
# Definición de las funciones de pérdida
# Estas funciones miden qué tan bien nuestras redes satisfacen la ecuación de onda,
# las condiciones iniciales y de frontera, y los datos de los sismogramas

# Se define la función de pérdida de la EDP
def wave_equation_residual(model_u, model_c, xzt):
    x = xzt[:, 0:1]
    z = xzt[:, 1:2]
    t = xzt[:, 2:3]
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch([x, z, t])
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch([x, z, t])
            u = model_u(tf.concat([x, z, t], axis=1))
        u_x = tape1.gradient(u, x)
        u_z = tape1.gradient(u, z)
        u_t = tape1.gradient(u, t)
    u_xx = tape2.gradient(u_x, x)
    u_zz = tape2.gradient(u_z, z)
    u_tt = tape2.gradient(u_t, t)

    # Se predice la velocidad c en los puntos dados
    c = model_c(tf.concat([x, z], axis=1))

    # Ecuación de onda: u_tt = c(x, y)^2 * (u_xx + u_zz)
    wave_residual = u_tt - c**2 * (u_xx + u_zz)

    # Se define Loss PDE (el término de la ecuación de onda en la función de pérdida)
    loss_pde = tf.reduce_mean(tf.square(wave_residual))

    return loss_pde

# Se define la función de pérdida de las condiciones iniciales
def IC_residual(model_u, xzt):
    x = np.random.uniform(x_min, x_max, (10000, 1))
    z = np.random.uniform(z_min, z_max, (10000, 1))
    t = np.random.uniform(t_min, t_max, (10000, 1))
    xzt = tf.convert_to_tensor(np.hstack((x, z, t)), dtype=tf.float32)
    x = xzt[:, 0:1]
    z = xzt[:, 1:2]
    t = xzt[:, 2:3]
    u_xz_0 = model_u(tf.concat([x, z, tf.zeros_like(t)], axis=1))

    # Condición inicial del desplazamiento
    ic_loss_1 = tf.reduce_mean(tf.square(u_xz_0 - f(x, z)))

    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, z, t])
        u = model_u(tf.concat([x, z, t], axis=1))
    u_t = tape.gradient(u, t)

    # Condición inicial de la velocidad
    mask_ic = tf.abs(t) < 1e-5
    ic_loss_2 = tf.reduce_mean(tf.where(mask_ic, tf.square(u_t), 0.0))

    return ic_loss_1 + ic_loss_2

# Se define la función de pérdida de las condiciones de frontera 
def BC_residual(model_u, model_c, xzt):
    x = xzt[:, 0:1]
    z = xzt[:, 1:2]
    t = xzt[:, 2:3]

    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch([x, z, t])
        with tf.GradientTape(persistent=True) as tape1:
            tape1.watch([x, z, t])
            u = model_u(tf.concat([x, z, t], axis=1))
        u_x = tape1.gradient(u, x)
        u_z = tape1.gradient(u, z)
        u_t = tape1.gradient(u, t)
    u_zz = tape2.gradient(u_z, z)

    c = model_c(tf.concat([x, z], axis=1))

    # Condición de la frontera superior (z=0)
    mask_top = tf.abs(z) < 1e-5
    bc_loss_top = tf.reduce_mean(tf.where(mask_top, tf.square(u_zz), 0.0))

    # Condiciones absorbentes para las otras fronteras
    mask_bottom = tf.abs(z - 1) < 1e-5
    bc_loss_bottom = tf.reduce_mean(tf.where(mask_bottom, tf.square(u_t + c * u_z), 0.0))

    mask_left = tf.abs(x) < 1e-5
    bc_loss_left = tf.reduce_mean(tf.where(mask_left, tf.square(u_t + c * u_x), 0.0))

    mask_right = tf.abs(x - 1) < 1e-5
    bc_loss_right = tf.reduce_mean(tf.where(mask_right, tf.square(u_t + c * u_x), 0.0))

    return bc_loss_top + bc_loss_bottom + bc_loss_left + bc_loss_right

# Se define la función de pérdida de los sismogramas
def SISM_residual(model_u, xzt):
    # Datos de los sismogramas obtenidos por DF
    loss_sism = 0.0

    # Se crea un vector de datos de tiempo iguales a t_sism
    t_values = np.linspace(0+t_sism[-1]/500, t_sism[-1], 500)

    for i in range(n):
        # Creamos una malla donde varia t conx y z constantes
        x = np.full_like(t_values, x_sism[i])  # x fijo en x_i
        z = np.full_like(t_values, 5e-5)  # z esta fijo en z=0

        # Convertimos x, z, t en un tensor para el modelo
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        z = tf.convert_to_tensor(z, dtype=tf.float32)
        t = tf.convert_to_tensor(t_values, dtype=tf.float32)

        xzt_tensor = tf.concat([x, z, t], axis=1)

        # Make predictions using the model
        u_pred_at_point = model_u(xzt_tensor)

        loss_sism += tf.reduce_mean(tf.abs(u_pred_at_point - tf.convert_to_tensor(u_sism[i].reshape(500, 1), dtype=tf.float32)))

    return loss_sism

# Función de pérdida
def loss_fn(model_u, model_c, xzt):
    # Loss EDP
    loss_pde = wave_equation_residual(model_u, model_c, xzt)

    # Loss condición inicial
    loss_ic = IC_residual(model_u, xzt)

    # Loss condición de frontera (Frontera libre: Frontera de Neumann)
    loss_bc = BC_residual(model_u, model_c, xzt) 

    # Loss Sismogramas
    loss_sism = SISM_residual(model_u, xzt)

    # Total loss
    # Pesos de cada término en la función de pérdida
    lambda_pde = 0.1 
    lambda_bc = 0.1 
    lambda_ic = 1.0
    lambda_sism = 1.0  

    total_loss = lambda_pde * loss_pde + lambda_sism * loss_sism + lambda_bc * loss_bc + lambda_ic * loss_ic
    return total_loss

# Optimizador: Adam 
adam = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4)

# Entrenamiento con ADAM
@tf.function
def train_step(model_u, model_c, xzt):
    with tf.GradientTape(persistent=True) as tape:
        loss = loss_fn(model_u, model_c, xzt)
    gradients_u = tape.gradient(loss, model_u.trainable_variables)
    gradients_c = tape.gradient(loss, model_c.trainable_variables)
    adam.apply_gradients(zip(gradients_u, model_u.trainable_variables))
    adam.apply_gradients(zip(gradients_c, model_c.trainable_variables))

    # Manualmente borramos el tape para salvar memoria
    del tape
    return loss


# Ciclo de entrenamiento
def train(model_u, model_c, xzt, epochs=1000):
    Total_loss=[]
    Loss_PDE=[]
    Ic_loss=[]
    Bc_loss=[]
    Loss_sism=[]
    # Entrenamiento con adam
    start = timeit.default_timer()
    for epoch in range(epochs):
        loss = train_step(model_u, model_c, xzt)
        Total_loss.append(loss.numpy())

        # Loss EDP
        loss_pde = wave_equation_residual(model_u, model_c, xzt)
        Loss_PDE.append(loss_pde.numpy())

        # Loss condición inicial
        loss_ic = IC_residual(model_u, xzt)
        Ic_loss.append(loss_ic.numpy())

        # Loss condición de frontera (free surface: Neumann boundary)
        loss_bc = BC_residual(model_u, model_c, xzt)  # Neumann boundary condition
        Bc_loss.append(loss_bc.numpy())

        # Loss Sismogramas
        loss_sism = SISM_residual(model_u, xzt)
        Loss_sism.append(loss_sism.numpy())

        if epoch % 10000 == 0:
            stop = timeit.default_timer()
            print('Time: ', stop - start)
            print(f'Epoch {epoch}, Loss: {loss.numpy()}, PDE Loss: {loss_pde.numpy()}, Initial Cond. Loss: {loss_ic.numpy()}, Boundary Loss: {loss_bc.numpy()}, Sismogram Loss: {loss_sism.numpy()}')
            start = timeit.default_timer()
            model_u.save(f'../Código Final/Modelos/Model_U_{epoch}')#, save_format="tf")
            model_c.save(f'../Código Final/Modelos/Model_V_{epoch}')#, save_format="tf")
            np.savez(f"/content/drive/MyDrive/PINNs - Inversión sismica/Terminos funcion de perdida/Terminos_Loss_{epoch}", Loss_PDE=Loss_PDE, Ic_loss=Ic_loss, Bc_loss=Bc_loss, Loss_sism=Loss_sism, Total_loss=Total_loss)

            x_test = np.linspace(0, 1, 100)
            z_test = np.linspace(0, 1, 100)
            X, Z = np.meshgrid(x_test, z_test)
            xz_test = tf.convert_to_tensor(np.hstack((X.reshape(-1, 1), Z.reshape(-1, 1))), dtype=tf.float32)
            v_pred = model_c(xz_test).numpy().reshape(100, 100)
            np.savez(f"../Código Final/Modelos/v_pred_{epoch}", v=v_pred)

            nelx = 100
            nely = 100
            timesteps = 101
            x = np.linspace(0,1,nelx+1)
            y = np.linspace(0,1,nely+1)
            xx,yy = np.meshgrid(x,y)
            x_ = np.zeros(shape = ((nelx+1) * (nely+1),))
            y_ = np.zeros(shape = ((nelx+1) * (nely+1),))
            for c1,ycor in enumerate(y):
                for c2,xcor in enumerate(x):
                    x_[c1*(nelx+1) + c2] = xcor
                    y_[c1*(nelx+1) + c2] = ycor
            X = np.column_stack((x_,y_))
            time=0.0
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X1 = np.column_stack((X,t_))
            T = model_u.predict(X1)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=0.1
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X2 = np.column_stack((X,t_))
            T = model_u.predict(X2)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=0.2
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X3 = np.column_stack((X,t_))
            T = model_u.predict(X3)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=0.3
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X4 = np.column_stack((X,t_))
            T = model_u.predict(X4)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=0.6
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X5 = np.column_stack((X,t_))
            T = model_u.predict(X5)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=1.0
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X6 = np.column_stack((X,t_))
            T = model_u.predict(X6)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)
            time=2.0
            t_ = np.ones((nelx+1) * (nely+1),) * (time)
            X7 = np.column_stack((X,t_))
            T = model_u.predict(X7)
            T = T.reshape(T.shape[0],)
            T = T.reshape(nelx+1,nely+1)
            np.savez(f"../Código Final/Modelos/u_pred_{epoch}_{time}", u=T)

    # Compute final losses
    final_loss = loss_fn(model_u, model_c, xzt)
    final_loss_pde = wave_equation_residual(model_u, model_c, xzt)
    final_loss_ic = IC_residual(model_u, xzt)
    final_loss_bc = BC_residual(model_u, model_c, xzt)
    final_loss_sism = SISM_residual(model_u, xzt)

    print(f'Final Loss: {final_loss.numpy()}, PDE Loss: {final_loss_pde.numpy()}, Initial Cond. Loss: {final_loss_ic.numpy()}, Boundary Loss: {final_loss_bc.numpy()}, Sismogram Loss: {final_loss_sism.numpy()}')

    return Loss_PDE, Ic_loss, Bc_loss, Loss_sism, Total_loss

In [ ]:
# Se entrena a la red
model_u = PINN()
model_c = VelocityModel()
Loss_PDE, Ic_loss, Bc_loss, Loss_sism, Total_loss=train(model_u, model_c, xzt, epochs=50000)

# Interpretación de resultados

Después del entrenamiento, analizamos los resultados de varias maneras:

1. **Gráfica de la función de pérdida**: Muestra cómo evolucionan los diferentes términos de la función de pérdida a lo largo del entrenamiento. Una disminución constante indica que la red está aprendiendo.

2. **Campo de velocidades predicho**: Comparamos el campo de velocidades predicho por la red con el campo verdadero (en este caso, homogéneo con v=0.5 km/s). Una predicción precisa debería mostrar un campo uniforme.

3. **Gráfica de la solución u(x,z,t)**: Nos permite visualizar la predicción de la onda en tres tiempos distintos.

4. **Animación de la solución u(x,z,t)**: Nos permite visualizar cómo la onda se propaga en el medio según la PINN. Debería mostrar un comportamiento físicamente plausible.

5. **Comparación de sismogramas**: Comparamos los sismogramas predichos por la PINN con los datos sintéticos originales. Una buena coincidencia indica que la PINN ha aprendido a reproducir los datos observados.

Estos análisis nos ayudan a evaluar qué tan bien la PINN ha aprendido a resolver la ecuación de onda y a invertir el campo de velocidades.

Términos de la función error en cada iteración

In [ ]:
# Figura de los términos de la función error
fig = plt.figure(1)
plt.plot(Ic_loss,'g',label='I.C')
plt.plot(Bc_loss,'black',label='B.C')
plt.plot(Total_loss,'--y',label='Total')
plt.plot(Loss_PDE,'r',label='PDE')
plt.plot(Loss_sism,'c',label='Seism')
plt.yscale("log")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.savefig('loss function.png',dpi=400)
plt.show()

#  Grafica de la predicción del campo de velocidades $v_{PINN}$(x,z)

In [ ]:
## Grafica de la predicción del campo de velocidad
x_test = np.linspace(0, 1, 100)
z_test = np.linspace(0, 1, 100)

X, Z = np.meshgrid(x_test, z_test)
xz_test = tf.convert_to_tensor(np.hstack((X.reshape(-1, 1), Z.reshape(-1, 1))), dtype=tf.float32)
v_pred = model_c(xz_test).numpy().reshape(100, 100)

plt.figure(2, figsize=(6, 4))
plt.contourf(X, Z, v_pred, levels=100, cmap='viridis')
plt.colorbar()# etiquetas
plt.title("Campo de velocidades")
plt.show()
# La velocidad en este caso es v=0.5, así que la imagen debería mostrar un campo de un solo color

In [ ]:
## Grafica
x_test = np.linspace(0, 1, 100)
z_test = np.linspace(0, 1, 100)
X, Z = np.meshgrid(x_test, z_test)
data = np.load("../Código Final/Modelos/v_pred2_50000.npz")
v_pred_1 = data["v"]

data = np.load("../Código Final/Modelos/v_pred2_0.npz")
v_pred_0 = data["v"]

# Pasos
Nx = 81        # Pasos en x
Nz = 81        # Pasos en y
v1=0.5
C2=np.ones((Nx+1,Nz+1))*v1
levels = np.linspace(0.0, 2.2, 2000)
# Soluciones por DF
fig, axs = plt.subplots(nrows=1,ncols=3, figsize=(14,4), sharex='col', sharey='row')
(ax1, ax2, ax3)= axs
# snapshot 1
cs1= ax1.contourf(x, z, C2, levels=levels, cmap="viridis")
ax1.invert_yaxis()
ax1.set_xlabel(r'Distancia x (km)', fontsize=12)
ax1.set_ylabel(r'Profundidad z (km)', fontsize=12)
ax1.set_aspect('equal', adjustable='box')
ax1.set_title("a) Verdadero campo v(x,z)")
# snapshot 2
cs2 = ax2.contourf(X, Z, v_pred_0, levels=levels, cmap='viridis')
ax2.invert_yaxis()
ax2.set_xlabel(r'Distancia x (km)', fontsize=12)
ax2.set_aspect('equal', adjustable='box')
ax2.set_title("b) Predicción inicial")
# snapshot 3
cs=ax3.contourf(X, Z, v_pred_1, levels=levels, cmap='viridis')
ax3.invert_yaxis()
ax3.set_xlabel(r'Distancia x (km)', fontsize=12)
ax3.set_aspect('equal', adjustable='box')
ax3.set_title("c) Resultado de la PINN")

plt.colorbar(cs, ax=axs, label='Velocidad (km/s)')
plt.savefig("Velocity field")
plt.show()

## Gráfica de la predicción de la solución $u_{PINN}$(x,z,t)

A continuación, se muestran tres graficas de la predicción de la PINN de u(x,z,t) en diferentes tiempos:
1. Al inicio (t=0.0 s)
2. En un tiempo intermedio (t=0.4 s)
3. Al final (t=6.0 s). Se utiliza para confirmar que las fronteras absorbieron a la onda y no hay más reflexiones.

Estas visualizaciones nos permiten observar si está prediciendo correctamente la propagación de la onda a través del dominio y su interacción con las fronteras.

In [ ]:
### Animación para u(x,z,t)
fig=plt.figure(101)
ax = fig.add_subplot(111)
nelx = 100
nely = 100
timesteps = 101
x = np.linspace(0,1,nelx+1)
y = np.linspace(0,1,nely+1)
t = np.linspace(0,4,timesteps)
delta_t = t[1] - t[0]
xx,yy = np.meshgrid(x,y)

x_ = np.zeros(shape = ((nelx+1) * (nely+1),))
y_ = np.zeros(shape = ((nelx+1) * (nely+1),))
for c1,ycor in enumerate(y):
    for c2,xcor in enumerate(x):
        x_[c1*(nelx+1) + c2] = xcor
        y_[c1*(nelx+1) + c2] = ycor
Ts = []

for time in t:
    t_ = np.ones((nelx+1) * (nely+1),) * (time)
    X = np.column_stack((x_,y_))
    X = np.column_stack((X,t_))
    T = model_u.predict(X)
    T = T.reshape(T.shape[0],)
    T = T.reshape(nelx+1,nely+1)
    Ts.append(T)
def plotmap(T,time):
    plt.clf()
    plt.title(f"Desplazamiento en t = {time*delta_t} s")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.pcolor(xx, yy, T, vmin=-0.1, vmax=0.1, cmap = 'RdBu')
    plt.colorbar()
    return plt

def animate(k):
    plotmap(Ts[k], k)

anim2 = animation.FuncAnimation(plt.figure(), animate, interval=10, frames=len(t), repeat=False)
anim2

# Animación de $u_{PINN}$(x, z, t)

In [ ]:
nelx = 100
nely = 100
timesteps = 101
x = np.linspace(0,1,nelx+1)
y = np.linspace(0,1,nely+1)
t = np.linspace(0,4,timesteps)
delta_t = t[1] - t[0]
xx,yy = np.meshgrid(x,y)

x_ = np.zeros(shape = ((nelx+1) * (nely+1),))
y_ = np.zeros(shape = ((nelx+1) * (nely+1),))
for c1,ycor in enumerate(y):
    for c2,xcor in enumerate(x):
        x_[c1*(nelx+1) + c2] = xcor
        y_[c1*(nelx+1) + c2] = ycor
X = np.column_stack((x_,y_))

time=0.0
t_ = np.ones((nelx+1) * (nely+1),) * (time)
X1 = np.column_stack((X,t_))
T = model_u.predict(X1)
T = T.reshape(T.shape[0],)
T = T.reshape(nelx+1,nely+1)

fig  = plt.figure(figsize = (16, 4))
# snapshot 1
ax   = fig.add_subplot(1, 3, 1, projection = "3d")
surf = ax.plot_surface(xx, yy, T, cmap = "coolwarm", linewidth = 0, vmin = -.5, vmax = .5)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(-1, 1)
ax.set_xlabel("x", fontstyle = "italic")
ax.set_ylabel("z", fontstyle = "italic")
ax.set_zlabel("u (x, z)", fontstyle = "italic")
ax.set_title(f"Posicion en t = {time} s")

# snapshot 2
time=0.4
t_ = np.ones((nelx+1) * (nely+1),) * (time)
X2 = np.column_stack((X,t_))
T = model_u.predict(X2)
T = T.reshape(T.shape[0],)
T = T.reshape(nelx+1,nely+1)

ax   = fig.add_subplot(1, 3, 2, projection = "3d")
surf = ax.plot_surface(xx, yy, T, cmap = "coolwarm", linewidth = 0, vmin = -.5, vmax = .5)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(-1, 1)
ax.set_xlabel("x", fontstyle = "italic")
ax.set_ylabel("z", fontstyle = "italic")
ax.set_zlabel("u (x, z)", fontstyle = "italic")
ax.set_title(f"Posicion en t = {time} s")

# snapshot 3
time=4.0
t_ = np.ones((nelx+1) * (nely+1),) * (time)
X3 = np.column_stack((X,t_))
T = model_u.predict(X3)
T = T.reshape(T.shape[0],)
T = T.reshape(nelx+1,nely+1)

ax   = fig.add_subplot(1, 3, 3, projection = "3d")
surf = ax.plot_surface(xx, yy, T, cmap = "coolwarm", linewidth = 0, vmin = -.5, vmax = .5)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_zlim(-1, 1)
ax.set_xlabel("x", fontstyle = "italic")
ax.set_ylabel("z", fontstyle = "italic")
ax.set_zlabel("u (x, z)", fontstyle = "italic")
ax.set_title(f"Posicion en t = {time} s")
plt.show()

# Gráfica comparativa de los sismogramas

In [ ]:
def plot_pinn_prediction(model, x_0, i,  z_0, t_max):
    """
    Grafica la predicción de la PINN para una coordenada espacial dada u(x_0, z_0, t).
    hasta el tiempo t_max

    Args:
        model: Trained PINN model.
        x_0: Valores de x_sism (posiciones de los receptores)
        i: Selección de uno de los receptores
        z_0: Valor fijo de z=z_0.
        t_max: El tiempo de cálculo de la predicción de la solución u(x,z,t) (p.ej., 6 segundos).
    """

    # Se crea un arreglo de puntos desde 0 hasta t_max
    t_values = np.linspace(0, t_max, 100)  # 100 time points from 0 to t_max

    # Se crean arreglos de igual tamaño para x y z
    x = np.full_like(t_values, x_0[i])  # x fija en x_0
    z = np.full_like(t_values, z_0)  # z fija en z_0

    # Se juntan y convierten en un tensor para el modelo
    xzt = np.vstack((x, z, t_values)).T
    xzt_tensor = tf.convert_to_tensor(xzt, dtype=tf.float32)

    # Se realiza la predicción
    u_predictions = model(xzt_tensor)
    result = u_predictions.items()

    # Convertir a una lista
    data = list(result)

    # Convertir la lista en un array
    numpyArray = np.array(data[0][1])

    # Se grafica la predicción junto al sismograma
    plt.plot(t_values, numpyArray+np.full_like(numpyArray, x_0[i]),"r-")
    plt.plot(t_sism,u_sism[i]+np.full_like(u_sism[i], x_0[i]),"k-")


# Ejemplo
fig = plt.figure(figsize=(6, 8))
plt.xlabel('t (s)', fontsize=15)
plt.ylabel('x (km)', fontsize=15)
plt.title("Sismograma del movimiento vertical", fontsize=18)
plt.legend()
plt.grid(True)
n=20
for i in range(n):
    plot_pinn_prediction(loaded, x_0=x_sism, i=i, z_0=0.0, t_max=6)

plt.show()